# 🔪 RecursiveCharacterTextSplitter - The Smart Text Splitter

## What is Text Splitting?

After loading documents, they're often **too large** for LLMs to process effectively. Text splitting breaks them into smaller, manageable **chunks**.

## Why RecursiveCharacterTextSplitter? 🌟

This is the **#1 recommended splitter** for generic text! Here's why:

### How It Works:
1. Tries to split on `"\n\n"` (paragraphs) first
2. If chunks are still too big → splits on `"\n"` (lines)
3. If still too big → splits on `" "` (words)
4. Last resort → splits on `""` (characters)

### Key Parameters:
| Parameter | Description | Example |
|-----------|-------------|---------|
| `chunk_size` | Maximum size of each chunk | 500 characters |
| `chunk_overlap` | Characters shared between chunks | 50 characters |
| `separators` | List of separators to try | `["\n\n", "\n", " ", ""]` |

### Why Overlap? 🔄
Overlap ensures **context is preserved** between chunks. Without it, sentences might be cut mid-way, losing meaning.

```
Chunk 1: [........text........]
Chunk 2:              [........text........]
                      ↑ overlap area ↑
```

📖 **Docs**: https://python.langchain.com/docs/how_to/recursive_text_splitter/

---

## 1️⃣ Splitting PDF Documents

Let's start by loading a PDF and then splitting it into chunks.

In [22]:
# Step 1: Load a PDF document first
from langchain_community.document_loaders import PyPDFLoader

# Load the "Attention Is All You Need" paper
loader = PyPDFLoader('attention.pdf')
docs = loader.load()

print(f"📄 Loaded {len(docs)} pages from PDF")
print(f"📏 First page length: {len(docs[0].page_content)} characters")

📄 Loaded 15 pages from PDF
📏 First page length: 2859 characters


In [23]:
# Each page is a Document object
print(f"Type: {type(docs[0])}")
print(f"\nDocument has two attributes:")
print(f"  • page_content: The actual text")
print(f"  • metadata: {docs[0].metadata}")

Type: <class 'langchain_core.documents.base.Document'>

Document has two attributes:
  • page_content: The actual text
  • metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}


## 2️⃣ Using RecursiveCharacterTextSplitter

Now let's split those PDF pages into smaller chunks!

In [24]:
# Import the text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create splitter with custom settings
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Each chunk max 500 characters
    chunk_overlap=50     # 50 character overlap between chunks
)

# Split the documents - this works directly with Document objects!
final_documents = text_splitter.split_documents(docs)

print(f"📊 Original: {len(docs)} pages")
print(f"📊 After splitting: {len(final_documents)} chunks")
print(f"📊 Average expansion: {len(final_documents)/len(docs):.1f}x")

📊 Original: 15 pages
📊 After splitting: 93 chunks
📊 Average expansion: 6.2x


In [25]:
# Compare first two chunks - notice they're smaller and may overlap
print("=" * 50)
print("📄 CHUNK 1:")
print("=" * 50)
print(f"Length: {len(final_documents[0].page_content)} chars")
print(f"Content: {final_documents[0].page_content[:300]}...")

print("\n" + "=" * 50)
print("📄 CHUNK 2:")
print("=" * 50)
print(f"Length: {len(final_documents[1].page_content)} chars")
print(f"Content: {final_documents[1].page_content[:300]}...")

📄 CHUNK 1:
Length: 492 chars
Content: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Par...

📄 CHUNK 2:
Length: 454 chars
Content: University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The ...


---

## 3️⃣ Splitting Plain Text Files

You can also split raw text strings using `create_documents()`.

In [26]:
# Load a text file using TextLoader
from langchain_community.document_loaders import TextLoader

loader = TextLoader('speech.txt')
docs = loader.load()

print(f"📄 Loaded document")
print(f"📏 Total length: {len(docs[0].page_content)} characters")
print(f"\n--- Preview ---")
print(docs[0].page_content[:500])

📄 Loaded document
📏 Total length: 3624 characters

--- Preview ---
The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.

Just because we fight withou


In [27]:
# Alternative: Read file directly and use create_documents()
# This is useful when you have raw strings instead of Document objects

with open("speech.txt") as f:
    speech = f.read()

# Create smaller chunks from raw text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,      # Smaller chunks for demo
    chunk_overlap=20     # 20 char overlap
)

# create_documents() takes a list of strings and returns Document objects
text_chunks = text_splitter.create_documents([speech])

print(f"📊 Created {len(text_chunks)} chunks from speech")
print(f"\n--- First Chunk ---")
print(f"Length: {len(text_chunks[0].page_content)} chars")
print(text_chunks[0].page_content)

print(f"\n--- Second Chunk ---")
print(f"Length: {len(text_chunks[1].page_content)} chars")
print(text_chunks[1].page_content)

📊 Created 49 chunks from speech

--- First Chunk ---
Length: 99 chars
The world must be made safe for democracy. Its peace must be planted upon the tested foundations of

--- Second Chunk ---
Length: 93 chars
foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no


In [28]:
# The result is still a Document object!
print(f"Type: {type(text_chunks[0])}")
print(f"\nYou can access:")
print(f"  • .page_content - the text")
print(f"  • .metadata - source info (empty for create_documents by default)")

Type: <class 'langchain_core.documents.base.Document'>

You can access:
  • .page_content - the text
  • .metadata - source info (empty for create_documents by default)


---

## 📝 Key Takeaways

### Two Ways to Split Text:

| Method | Input | Use When |
|--------|-------|----------|
| `split_documents(docs)` | List of Document objects | After using a DocumentLoader |
| `create_documents([strings])` | List of raw strings | When you have plain text |

### Choosing chunk_size and chunk_overlap:

- **chunk_size**: Depends on your LLM's context window
  - Smaller (100-500): More precise retrieval, less context
  - Larger (500-2000): More context, may include irrelevant info

- **chunk_overlap**: Usually 10-20% of chunk_size
  - Too small: May lose context at boundaries
  - Too large: Redundant information, more chunks

### Best Practices 💡

1. ✅ Use RecursiveCharacterTextSplitter for general text
2. ✅ Test different chunk sizes for your use case
3. ✅ Keep overlap for context preservation
4. ✅ Consider your embedding model's token limits

### Next Steps 🚀
- Learn about **CharacterTextSplitter** for simple splitting
- Explore **HTMLTextSplitter** for web content
- Try **RecursiveJsonSplitter** for JSON data